In [2]:
from functions_py import convert_to_polar, polar_conversion, func_def, fourier_coef_calc, get_label, img_plot, polar_conversion_fast, get_label_fast
from functions_py import pipeline_train, batch_get_labels, optimized_test_class_alg
from functions_py import optimized_func_def, optimized_fourier_coef_calc
from functions_py import qwen_pipeline_train, qwen_optimized_test_class_alg, qwen_optimized_func_def, qwen_optimized_fourier_coef_calc, qwen_polar_conversion_fast
from functions_py import run_hyperparameter_optimization
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from numba import njit
from scipy.spatial.distance import cdist


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import os

# Функции предобработки
def transform_pixels_to_sums_vectorized(df):
    # Создаем копию исходного DataFrame
    result_df = df[['label']].copy()
    
    # Получаем все пиксельные колонки
    pixel_cols = [f'pixel{i}' for i in range(784)]
    pixels = df[pixel_cols].values
    
    # Преобразуем в 3D массив (n_images, 28, 28)
    images = pixels.reshape(-1, 28, 28)
    
    # Суммы по строкам (axis=2 - сумма по столбцам в каждом ряду)
    row_sums = images.sum(axis=2)
    
    # Суммы по столбцам (axis=1 - сумма по рядам в каждом столбце)
    col_sums = images.sum(axis=1)
    
    # Добавляем суммы в DataFrame
    for i in range(28):
        result_df[f'sum_row{i}'] = row_sums[:, i] / 28
    for i in range(28):
        result_df[f'sum_column{i}'] = col_sums[:, i] / 28
    
    return result_df

# вычисление аккураси
def test_train_class(predobr_func, D, M, T):
    base = ''
    input_data = pd.read_csv(base + 'train.csv')
    conversion_state = True
    write_state = False
    state = 148
    target = 'label'
    data_with_labels = predobr_func(input_data)
    
    t_coef = T

    
    data_train, data_test, data_train_means, fourier_coef = qwen_pipeline_train(data_with_labels, target, state, conversion_state, write_state, D, M)
    print('train acc')
    accuracy = qwen_optimized_test_class_alg(target,data_train , data_train, data_train_means, fourier_coef, t_coef, rewrite = True)
    print(accuracy)
    print('test acc')
    accuracy = qwen_optimized_test_class_alg(target,data_test , data_train, data_train_means, fourier_coef, t_coef, rewrite = True)
    print(accuracy)
    return data_train, data_test, data_train_means, fourier_coef

In [3]:
data_train, data_test, data_train_means, fourier_coef = test_train_class(transform_pixels_to_sums_vectorized, 13983, 29, 1.5847369)

train acc
0.712544420753376
test acc
0.6932178932178932


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.base import ClassifierMixin
from sklearn.model_selection import train_test_split

#вывод изображения
def img_plot(img):
  plt.figure(figsize=(7,7))

  grid_data = img.values.reshape(28,28)
  plt.imshow(grid_data,interpolation= "none" , cmap="gray")
  plt.show()


base = ''
input_data = pd.read_csv(base + 'train.csv')
state = 148
target = 'label'
temp_train, temp_test = train_test_split(input_data, stratify=input_data[target], test_size=0.33, random_state=state) 

idx = 3
image = temp_test.iloc[idx:idx+1]
# img_plot(image.drop(columns=[target]))
t = 1.5847369
img_transformed = transform_pixels_to_sums_vectorized(image).drop(columns=[target]).values
res = batch_get_labels(
        target, data_train, data_train_means, fourier_coef, img_transformed, t, is_labeled = False
    )

dolya = res / 56 * 100
print([round(x, 2) for x in dolya[0]])
print(np.argmax(res, axis=1))


[37.5, 39.29, 48.21, 48.21, 35.71, 44.64, 41.07, 41.07, 42.86, 35.71]
[2]


In [31]:
from train_test import rec_digit
tmpdd = image.drop(columns=[target]).values.reshape(28,28)
tmpdd = np.uint8(tmpdd)
# print(image.drop(columns=[target]).values.reshape(28,28).shape)

res = rec_digit(tmpdd)
print((res * 100).tolist())
print(int(res.argmax()))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[99.99821472167969, 8.317831779594177e-21, 8.072601254127676e-09, 5.578918471399637e-14, 7.783363363544904e-12, 2.3757544909130957e-07, 2.990961298077188e-11, 3.6889137179827954e-16, 0.0017880876548588276, 9.18854770048938e-09]]
0
